Install and import neccersary packages

In [2]:
!pip install pandas
!pip install nltk
!pip install stop-words
from stop_words import get_stop_words
import nltk
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import IPython.display as display
import matplotlib.pyplot as plt
import math
import random
import time as time
from transformers import BertTokenizerFast, TFBertModel, TFBertPreTrainedModel, TFBertMainLayer
from tensorflow import keras
import pathlib
import jsonlines
from transformers import AutoTokenizer, AutoModelWithLMHead, TFAutoModelWithLMHead, TFAutoModel
import re
import itertools
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
import pandas as pd

Defaulting to user installation because normal site-packages is not writeable


Load the data into a pandas dataframe

In [7]:
df = pd.read_json('NQAltered.json')

In [8]:
df.head()

id                                                url  \
0 -7192501076945116903  https://en.wikipedia.org//w/index.php?title=Mi...   
1  4515227776076825962  https://en.wikipedia.org//w/index.php?title=Re...   
2  1744559614254480616  https://en.wikipedia.org//w/index.php?title=Po...   
3  1769260702241487320  https://en.wikipedia.org//w/index.php?title=Is...   
4 -6150633611189338412  https://en.wikipedia.org//w/index.php?title=Tu...   

                                       question_text  \
0     who plays mike in how to lose a guy in 10 days   
1  disorders of the reproductive system male and ...   
2  what major event challenged the universal powe...   
3   where does the isle of wight festival take place   
4    afrojack & martin garrix - turn up the speakers   

                                       original_text  answer    answer_loc  
0  Mike Bassett : England Manager - wikipedia <H1...    True  [1561, 1733]  
1  Reproductive system disease - wikipedia <H1> R...   False      [-1, -1]  
2  Pope - wikipedia <H1> Pope </H1> Jump to : nav...   False      [-1, -1]  
3  Isle of Wight Festival - wikipedia <H1> Isle o...    True    [113, 151]  
4  Turn Up the Speakers - Wikipedia <H1> Turn Up ...   False      [-1, -1]

In [14]:
df.iloc[:40000, :].shape

(40000, 6)

Model A1: Always predict answer

In [19]:
total = 40000
correct = 0
tp, fp, tn, fn = 0, 0, 0, 0
for index, row in df.iloc[:40000, :].iterrows():
    if row['answer'] is True:
        correct += 1
        tp += 1
    else:
        fp += 1

In [20]:
prec = tp/(tp+fp)
rec = tp/(tp+fn)
f1 = 2*((prec*rec)/(prec+rec))
print("accuracy", correct/total, "precision", prec, "recall", rec, "F1", f1)

accuracy 0.4921 precision 0.4921 recall 1.0 F1 0.6596072649286241


Model A2: Predict based on percent words from question in document after removing stop words

In [53]:
#First find optimal threshold
acc_correct_ans = 0
acc_wrong_ans = 0
num_correct_ans = 0
num_wrong_ans = 0
stop_words = set(get_stop_words('english'))  
for index, row in df.iloc[:40000, :].iterrows():
    filtered_question = [w for w in row['question_text'].split(' ') if not w in stop_words] 
    text = set([r.lower() for r in row['original_text'].split(" ")])
    c = sum(el in text for el in filtered_question)
    if row['answer'] is True and len(filtered_question) != 0:
        num_correct_ans += 1
        acc_correct_ans += (c/len(filtered_question))
    elif len(filtered_question) != 0:
        num_wrong_ans += 1
        acc_wrong_ans += (c/len(filtered_question))
print("Average right", acc_correct_ans/num_correct_ans, "Average wrong", acc_wrong_ans/num_wrong_ans)

Average right 0.8670220729900875 Average wrong 0.8383439839346584


In [56]:
threshold = .85
total = 40000
correct = 0
tp, fp, tn, fn = 0, 0, 0, 0
for index, row in df.iloc[:40000, :].iterrows():
    filtered_question = [w for w in row['question_text'].split(' ') if not w in stop_words] 
    text = set([r.lower() for r in row['original_text'].split(" ")])
    c = sum(el in text for el in filtered_question)
    perc = c / len(filtered_question) if len(filtered_question) != 0 else 0
    if perc >= threshold and row['answer'] is True:
        correct += 1
        tp += 1
    if perc >= threshold and row['answer'] is False: fp += 1
    if perc < threshold and row['answer'] is True: fn += 1
    if perc < threshold and row['answer'] is False:
        tn += 1
        correct += 1

In [57]:
prec = tp/(tp+fp)
rec = tp/(tp+fn)
f1 = 2*((prec*rec)/(prec+rec))
print("accuracy", correct/total, "precision", prec, "recall", rec, "F1", f1)

accuracy 0.523775 precision 0.5152300091140212 recall 0.5456716114610851 F1 0.5300140633095655
